### First capstone project
1. how does age by various citations change throughout the year?
1. how does age within specific citations change throughout the year?
    - older people with winter homes during the cooler months
1. time of day, mean age of person getting citation
    - younger people on their way to work getting speeding tickets
1. time of day which citations are happening more than others
    - younger people out later
1. gender of the people during time of day

next steps:
1. get more date and compare year over year
2. 

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime
import glob
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
# pd.set_option('display.max_columns', None)

In [9]:
# read in csv files
files = glob.glob('data/*.csv')

citations = pd.concat([pd.read_csv(f) \
                      for f in files], sort=True)

In [11]:
# convert column labels to lower case and remove spaces from column names
def setcols(df, fn=lambda x: x.columns.map('_'.join), cols=None):
    if cols:
        df.columns = cols
    else:
        df.columns = fn(df)
    return df

citations = citations.pipe(
        setcols,
        fn=lambda x: x.columns.str.lower()
        .str.strip()
        .str.replace(" ", "_"))

In [57]:
citations.head()

,arizona_statute_code,beat,charge_description,citation_#,citation_date,citation_time,citation_type_code,citation_type_description,cited_person,cited_person_age,...,state,street,street_number,zip,zone,tcmainid,year_month,month,year,hour_of_day
0,97XX,12.0,Amended Charge,2074492,2018-12-20,2222,CV,Civil Traffic,"Way, Dennis",51,...,AZ,E Sahuaro DR / N Scottsdale Rd,xxx,NaN,3003,641301,2018.12,12,2018,22
1,28-701A,10.0,Speed Greater Than R&P or Posted,8093305,2018-12-20,844,CV,Civil Traffic,"Grady, Scott",61,...,AZ,N Scottsdale Rd,8xxx,NaN,2601,641302,2018.12,12,2018,8
2,28-4135C,3.0,No Proof of Insurance,2076259,2018-12-18,2147,CV,Civil Traffic,"Dudgeon, Miranda",20,...,AZ,E Osborn Rd / N Scottsdale Rd,xxx,NaN,706,641303,2018.12,12,2018,21
3,28-2532A,5.0,Expired/No Arizona Registration,8093703,2018-12-21,2116,CV,Civil Traffic,"Fox, William",34,...,AZ,E Chaparral Rd / N Hayden Rd,xxx,NaN,1106,641304,2018.12,12,2018,21
4,28-4135C,15.0,No Proof of Insurance,2053182,2018-12-20,2324,CV,Civil Traffic,"Rhoden, David",24,...,AZ,E Frank Lloyd Wright Bl / N Hayden Rd,xxx,NaN,3604,641305,2018.12,12,2018,23


In [13]:
# remove duplicate citations due to overlap in data being downloaded
citations.drop_duplicates('citation_#', inplace=True)

In [14]:
# convert citation date to datetime format
citations['citation_date'] = pd.to_datetime(citations['citation_date'])

In [15]:
# combine both month and year into same column
citations['year_month'] = citations['citation_date'].apply(lambda x: str(x.year) + '.' + str(x.month).zfill(2))

In [16]:
# create month column
citations['month'] = citations['citation_date'].dt.month

# create year column
citations['year'] = citations['citation_date'].dt.year

In [17]:
# strip out whitespace
# citations = citations.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [56]:
citations['hour_of_day'] = citations['citation_time']. \
    apply(lambda x: str(x)[-4:-2]) # extract the hour of the day from the citation time column

In [92]:
# replace blanks with 0
# this happened for citations that took place during the hour of 12am
citations.loc[citations.hour_of_day == '', 'hour_of_day'] = 0

# why doesn't the below work?
# citations.loc[citations.hour_of_day == '', citations.hour_of_day]


In [96]:
# change data type of hour from object to number
citations.hour_of_day = pd.to_numeric(citations.hour_of_day)

In [76]:
# change data type of age from object to number
citations.cited_person_age = pd.to_numeric(citations.cited_person_age, errors='coerce')

In [34]:
citations.cited_person_age.value_counts().head()

23.0    1881
22.0    1820
24.0    1815
25.0    1652
21.0    1596
Name: cited_person_age, dtype: int64

In [97]:
citations.head()

,arizona_statute_code,beat,charge_description,citation_#,citation_date,citation_time,citation_type_code,citation_type_description,cited_person,cited_person_age,...,state,street,street_number,zip,zone,tcmainid,year_month,month,year,hour_of_day
0,97XX,12.0,Amended Charge,2074492,2018-12-20,2222,CV,Civil Traffic,"Way, Dennis",51.0,...,AZ,E Sahuaro DR / N Scottsdale Rd,xxx,NaN,3003,641301,2018.12,12,2018,22
1,28-701A,10.0,Speed Greater Than R&P or Posted,8093305,2018-12-20,844,CV,Civil Traffic,"Grady, Scott",61.0,...,AZ,N Scottsdale Rd,8xxx,NaN,2601,641302,2018.12,12,2018,8
2,28-4135C,3.0,No Proof of Insurance,2076259,2018-12-18,2147,CV,Civil Traffic,"Dudgeon, Miranda",20.0,...,AZ,E Osborn Rd / N Scottsdale Rd,xxx,NaN,706,641303,2018.12,12,2018,21
3,28-2532A,5.0,Expired/No Arizona Registration,8093703,2018-12-21,2116,CV,Civil Traffic,"Fox, William",34.0,...,AZ,E Chaparral Rd / N Hayden Rd,xxx,NaN,1106,641304,2018.12,12,2018,21
4,28-4135C,15.0,No Proof of Insurance,2053182,2018-12-20,2324,CV,Civil Traffic,"Rhoden, David",24.0,...,AZ,E Frank Lloyd Wright Bl / N Hayden Rd,xxx,NaN,3604,641305,2018.12,12,2018,23


In [15]:
citations.shape

(44383, 25)

In [98]:
citations.dtypes

arizona_statute_code                 object
beat                                float64
charge_description                   object
citation_#                            int64
citation_date                datetime64[ns]
citation_time                         int64
citation_type_code                   object
citation_type_description            object
cited_person                         object
cited_person_age                    float64
cited_person_ethnicity               object
cited_person_race                    object
cited_person_sex                     object
city                                 object
district                             object
officer_badge_#                      object
state                                object
street                               object
street_number                        object
zip                                  object
zone                                 object
tcmainid                              int64
year_month                      

In [12]:
# grab the top 5 citations, excluding speeding since it's in a league of its own
citations.charge_description.value_counts()[1:6].index

Index(['Expired/No Arizona Registration', 'Suspended/Revoked D.L.',
       'Failure to Control Speed to Avoid A Collision',
       'No Proof of Insurance', 'DUI-Impaired to Slightest Degree'],
      dtype='object')

In [104]:
# grab the top 20 citations, excluding speeding since it's in a league of its own
citations.charge_description.value_counts()[1:21].index

Index(['Expired/No Arizona Registration', 'Suspended/Revoked D.L.',
       'Failure to Control Speed to Avoid A Collision',
       'No Proof of Insurance', 'DUI-Impaired to Slightest Degree',
       'Drug Paraphernalia-Possess/Use', 'Fail to Obey Traffic Control Device',
       'Fail to Stop at Stop Sign', 'Fail to Stop For Red Signal',
       'Exceed Limit By More Than 20mph', 'Exceed 15mph In School Crossing',
       'Urinate/Defecate In Public', 'Void Charge',
       'Fail to Yield Turning Left at Intersection',
       'Shoplifting-Removal of Goods', 'Unsafe Lane Change',
       'Display M.I. Suspended Plate', 'Expired Out-Of-State Registration',
       'Disorderly Conduct-Disruptive Behavior',
       'No Arizona or M/C Driver`s License'],
      dtype='object')

In [58]:
citations.charge_description.value_counts().head(10)

Speed Greater Than R&P or Posted                 10689
Expired/No Arizona Registration                   3602
Suspended/Revoked D.L.                            2806
Failure to Control Speed to Avoid A Collision     2108
No Proof of Insurance                             2091
DUI-Impaired to Slightest Degree                  2054
Drug Paraphernalia-Possess/Use                    2005
Fail to Obey Traffic Control Device               1453
Fail to Stop at Stop Sign                         1282
Fail to Stop For Red Signal                       1194
Name: charge_description, dtype: int64

In [105]:
top5_citations = citations[citations.charge_description.isin(citations.charge_description.value_counts()[1:6].index)]

top20_citations = citations[citations.charge_description.isin(citations.charge_description.value_counts()[1:21].index)]

In [79]:
top5_citations.head()

,arizona_statute_code,beat,charge_description,citation_#,citation_date,citation_time,citation_type_code,citation_type_description,cited_person,cited_person_age,...,state,street,street_number,zip,zone,tcmainid,year_month,month,year,hour_of_day
2,28-4135C,3.0,No Proof of Insurance,2076259,2018-12-18,2147,CV,Civil Traffic,"Dudgeon, Miranda",20.0,...,AZ,E Osborn Rd / N Scottsdale Rd,xxx,NaN,706,641303,2018.12,12,2018,21
3,28-2532A,5.0,Expired/No Arizona Registration,8093703,2018-12-21,2116,CV,Civil Traffic,"Fox, William",34.0,...,AZ,E Chaparral Rd / N Hayden Rd,xxx,NaN,1106,641304,2018.12,12,2018,21
4,28-4135C,15.0,No Proof of Insurance,2053182,2018-12-20,2324,CV,Civil Traffic,"Rhoden, David",24.0,...,AZ,E Frank Lloyd Wright Bl / N Hayden Rd,xxx,NaN,3604,641305,2018.12,12,2018,23
13,28-2532A,7.0,Expired/No Arizona Registration,2077577,2018-12-19,2106,CV,Civil Traffic,"Rossman, Michael",56.0,...,AZ,E Indian School Rd / N Scottsdale Rd,xxx,NaN,805,641316,2018.12,12,2018,21
16,28-2532A,10.0,Expired/No Arizona Registration,8092798,2018-12-20,2118,CV,Civil Traffic,"Jarvis, Sidney",31.0,...,AZ,N Scottsdale Rd,7xxx,NaN,2501,641319,2018.12,12,2018,21


## Month of the year and age analysis

In [50]:
top5_citations.groupby([top5_citations.month, top5_citations.charge_description])['cited_person_age'].mean().unstack()

charge_description,DUI-Impaired to Slightest Degree,Expired/No Arizona Registration,Failure to Control Speed to Avoid A Collision,No Proof of Insurance,Suspended/Revoked D.L.
month,,,,,
1,35.707447,36.895442,43.662551,37.637255,34.561947
2,35.487013,38.296820,40.538835,37.375000,34.638095
3,35.436893,38.303030,42.476415,37.753623,33.781132
4,35.052632,37.294985,40.157609,38.403315,34.954545
5,33.157407,36.545732,39.619792,35.820755,34.325000
6,33.430108,37.385496,38.798701,34.343066,33.825472
7,32.128713,35.777778,36.081081,35.480315,34.518293
8,34.294118,38.694301,37.529412,34.278846,34.239726
9,34.187500,37.673387,37.883333,35.292857,34.656977


In [52]:
top5_citations.groupby([top5_citations.month, top5_citations.charge_description])['cited_person_age'].mean().unstack().loc[:,"DUI-Impaired to Slightest Degree"]

month
1     35.707447
2     35.487013
3     35.436893
4     35.052632
5     33.157407
6     33.430108
7     32.128713
8     34.294118
9     34.187500
10    35.416185
11    34.763006
12    32.492126
Name: DUI-Impaired to Slightest Degree, dtype: float64

In [53]:
top5_citations.groupby([top5_citations.month, top5_citations.charge_description])['cited_person_age'].mean().unstack().loc[:,"Failure to Control Speed to Avoid A Collision"]

month
1     43.662551
2     40.538835
3     42.476415
4     40.157609
5     39.619792
6     38.798701
7     36.081081
8     37.529412
9     37.883333
10    39.896226
11    42.179245
12    42.553299
Name: Failure to Control Speed to Avoid A Collision, dtype: float64

In [58]:
from scipy.stats import ttest_ind

ttest_ind(top5_citations.groupby([top5_citations.month, top5_citations.charge_description])['cited_person_age'] \
          .mean().unstack().loc[:,"DUI-Impaired to Slightest Degree"],\
          top5_citations.groupby([top5_citations.month, top5_citations.charge_description])['cited_person_age'] \
          .mean().unstack().loc[:,"Failure to Control Speed to Avoid A Collision"])

Ttest_indResult(statistic=-7.7048997378042134, pvalue=1.0953952602151523e-07)

### I would guess that the below two are more similar throughout the year than above two

In [59]:
ttest_ind(top5_citations.groupby([top5_citations.month, top5_citations.charge_description])['cited_person_age'] \
          .mean().unstack().loc[:,"DUI-Impaired to Slightest Degree"],\
          top5_citations.groupby([top5_citations.month, top5_citations.charge_description])['cited_person_age'] \
          .mean().unstack().loc[:,"No Proof of Insurance"])

Ttest_indResult(statistic=-3.610039753505458, pvalue=0.0015534291479321221)

## Hour of the day and age analysis

### top 5 citations

In [100]:
top5_citations.groupby([top5_citations.hour_of_day, top5_citations.charge_description])['cited_person_age'].mean().unstack()

charge_description,DUI-Impaired to Slightest Degree,Expired/No Arizona Registration,Failure to Control Speed to Avoid A Collision,No Proof of Insurance,Suspended/Revoked D.L.
hour_of_day,,,,,
0,33.208191,32.316667,35.363636,33.956204,33.142157
1,31.358974,32.236842,34.714286,32.552941,33.242188
2,29.117816,33.846154,28.285714,33.780000,32.435294
3,29.435897,34.529412,21.333333,32.764706,33.146341
4,32.230769,43.000000,40.666667,38.500000,36.862069
5,28.625000,33.900000,35.214286,36.857143,34.935484
6,28.666667,36.000000,41.645161,35.250000,37.137931
7,36.666667,38.917431,40.397849,39.478261,34.975904
8,40.625000,37.037234,39.051471,38.237179,34.964912


### top 20 citations

In [107]:
top20_citations.groupby([top20_citations.hour_of_day, top20_citations.charge_description])['cited_person_age'].mean().unstack()

charge_description,DUI-Impaired to Slightest Degree,Disorderly Conduct-Disruptive Behavior,Display M.I. Suspended Plate,Drug Paraphernalia-Possess/Use,Exceed 15mph In School Crossing,Exceed Limit By More Than 20mph,Expired Out-Of-State Registration,Expired/No Arizona Registration,Fail to Obey Traffic Control Device,Fail to Stop For Red Signal,Fail to Stop at Stop Sign,Fail to Yield Turning Left at Intersection,Failure to Control Speed to Avoid A Collision,No Arizona or M/C Driver`s License,No Proof of Insurance,Shoplifting-Removal of Goods,Suspended/Revoked D.L.,Unsafe Lane Change,Urinate/Defecate In Public,Void Charge
hour_of_day,,,,,,,,,,,,,,,,,,,,
0,33.208191,26.861111,32.323529,29.626794,NaN,28.500000,32.461538,32.316667,34.450980,33.388060,33.095238,32.000000,35.363636,34.285714,33.956204,31.500000,33.142157,28.000000,26.692771,35.833333
1,31.358974,28.000000,32.333333,28.291391,NaN,29.200000,35.000000,32.236842,37.965517,30.800000,30.450000,51.000000,34.714286,28.333333,32.552941,42.333333,33.242188,38.000000,26.943299,34.263158
2,29.117816,27.303030,33.909091,28.885246,NaN,28.750000,27.000000,33.846154,35.125000,31.692308,40.000000,32.000000,28.285714,31.666667,33.780000,33.000000,32.435294,30.000000,28.047337,29.300000
3,29.435897,27.000000,38.500000,30.277778,NaN,24.000000,22.000000,34.529412,31.857143,35.428571,38.000000,72.000000,21.333333,40.000000,32.764706,26.000000,33.146341,NaN,25.666667,39.666667
4,32.230769,28.600000,35.750000,32.030303,NaN,32.833333,NaN,43.000000,52.800000,39.625000,23.000000,61.000000,40.666667,41.500000,38.500000,27.000000,36.862069,NaN,NaN,24.666667
5,28.625000,31.000000,36.833333,31.600000,NaN,34.062500,32.000000,33.900000,43.583333,40.090909,44.250000,48.333333,35.214286,38.333333,36.857143,29.000000,34.935484,42.666667,44.000000,24.333333
6,28.666667,27.000000,46.000000,34.130435,59.000000,33.576923,28.600000,36.000000,42.421053,50.607143,40.285714,40.400000,41.645161,39.600000,35.250000,52.000000,37.137931,43.800000,NaN,35.500000
7,36.666667,46.444444,33.076923,34.837838,45.517241,29.545455,34.000000,38.917431,42.808081,40.486486,44.078947,41.103448,40.397849,38.875000,39.478261,34.833333,34.975904,46.391304,NaN,43.750000
8,40.625000,38.250000,29.333333,32.684211,41.655000,37.012987,35.333333,37.037234,40.807018,45.000000,42.079365,43.038462,39.051471,34.766667,38.237179,34.714286,34.964912,45.600000,NaN,35.647059
